In [ ]:
%load_ext nb_black

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv(r'C:\Users\ventu\pyProjects\Data_Science_Projects\Used Phones\storage.csv',index_col=[0])
df.head()

In [ ]:
df.columns

In [ ]:
# df = pd.read_csv('/home/venturi/Projects/Data_Science_Projects/Used Phones/storage.csv')

In [ ]:
df.set_index('Listing')

In [ ]:
df = df.drop('Unnamed: 0', axis=1)

In [ ]:
df.to_csv('/home/venturi/Projects/Data_Science_Projects/Used Phones/storage.csv')

In [ ]:
df["Price"] = pd.to_numeric(df["Price"])
df.to_csv('/home/venturi/Projects/Data_Science_Projects/Used Phones/storage.csv')

In [ ]:
df2 = df.loc[df['Sold']=='Yes'].groupby('Model').Price.mean()

In [ ]:
df2.value

In [ ]:
import plotly.graph_objects as go

fig = go.Figure(go.Bar(
    orientation="h",
    x = df2.values,
    y = df2.index))

fig.update_layout(
    autosize=False,
    height=600,
    yaxis={'categoryorder':'total ascending'},
)

fig.show()

In [ ]:
import plotly.graph_objects as go

fig = go.Figure(go.Bar(
            x=[20, 14, 23],
            y=['giraffes', 'orangutans', 'monkeys']))

fig.show()

In [ ]:
df3 = pd.read_csv('/home/venturi/Projects/Data_Science_Projects/Used Phones/storage.csv',usecols=[0,1,2,3])
df3.head()
